# 模拟环境：Gymnasium

对于 LLM 代理的许多应用来说，环境是真实的（互联网、数据库、REPL 等）。然而，我们也可以定义代理在模拟环境中进行交互，比如基于文本的游戏。这是一个如何使用 [Gymnasium](https://github.com/Farama-Foundation/Gymnasium)（前身是 [OpenAI Gym](https://github.com/openai/gym)）创建简单的代理-环境交互循环的示例。

In [1]:
!pip install gymnasium

In [2]:
import tenacity
from langchain.output_parsers import RegexParser
from langchain.schema import (
    HumanMessage,
    SystemMessage,
)

## 定义代理

In [ ]:
class GymnasiumAgent:
    @classmethod
    def get_docs(cls, env):
        return env.unwrapped.__doc__

    def __init__(self, model, env):
        self.model = model
        self.env = env
        self.docs = self.get_docs(env)

        self.instructions = """
你的目标是最大化你的回报，即你收到的奖励总和。
我会给你一个观察值、奖励、终止标志、截断标志和到目前为止的回报，格式如下：

观察值：<observation>
奖励：<reward>
终止：<termination>
截断：<truncation>
回报：<sum_of_rewards>

你将用以下格式回应一个动作：

动作：<action>

其中 <action> 替换为你的实际动作。
除了返回动作外不要做任何其他事情。
"""
        self.action_parser = RegexParser(
            regex=r"Action: (.*)", output_keys=["action"], default_output_key="action"
        )

        self.message_history = []
        self.ret = 0

    def random_action(self):
        action = self.env.action_space.sample()
        return action

    def reset(self):
        self.message_history = [
            SystemMessage(content=self.docs),
            SystemMessage(content=self.instructions),
        ]

    def observe(self, obs, rew=0, term=False, trunc=False, info=None):
        self.ret += rew

        obs_message = f"""
观察值：{obs}
奖励：{rew}
终止：{term}
截断：{trunc}
回报：{self.ret}
        """
        self.message_history.append(HumanMessage(content=obs_message))
        return obs_message

    def _act(self):
        act_message = self.model.invoke(self.message_history)
        self.message_history.append(act_message)
        action = int(self.action_parser.parse(act_message.content)["action"])
        return action

    def act(self):
        try:
            for attempt in tenacity.Retrying(
                stop=tenacity.stop_after_attempt(2),
                wait=tenacity.wait_none(),  # 重试之间没有等待时间
                retry=tenacity.retry_if_exception_type(ValueError),
                before_sleep=lambda retry_state: print(
                    f"发生了 ValueError：{retry_state.outcome.exception()}，正在重试..."
                ),
            ):
                with attempt:
                    action = self._act()
        except tenacity.RetryError:
            action = self.random_action()
        return action

## 初始化模拟环境和代理

In [ ]:
import gymnasium as gym  # 导入 gymnasium 并重命名为 gym
from langchain_openai import ChatOpenAI  # 导入 ChatOpenAI 模型

env = gym.make("Blackjack-v1")  # 创建二十一点游戏环境
agent = GymnasiumAgent(model=ChatOpenAI(temperature=0.2), env=env)  # 初始化游戏代理

## 主循环

In [ ]:
# 主循环开始
observation, info = env.reset()  # 重置环境
agent.reset()  # 重置代理

obs_message = agent.observe(observation)  # 代理观察初始状态
print(obs_message)

while True:
    action = agent.act()  # 代理选择动作
    observation, reward, termination, truncation, info = env.step(action)  # 执行动作
    obs_message = agent.observe(observation, reward, termination, truncation, info)  # 代理观察新状态
    print(f"动作：{action}")
    print(obs_message)

    if termination or truncation:  # 如果游戏结束或达到最大步数
        print("结束", termination, truncation)
        break
env.close()  # 关闭环境


Observation: (15, 4, 0)
Reward: 0
Termination: False
Truncation: False
Return: 0
        
Action: 1

Observation: (25, 4, 0)
Reward: -1.0
Termination: True
Truncation: False
Return: -1.0
        
break True False
